In [ ]:
Name-Atharva Talegaonkar SID-862467125

In [7]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17) 
  
# data (as pandas dataframes) 
X = breast_cancer_wisconsin_diagnostic.data.features.to_numpy()
y = breast_cancer_wisconsin_diagnostic.data.targets.to_numpy()
  
print(type(X))

<class 'numpy.ndarray'>


In [ ]:
Question -1  Implementing simple classifiers

Here i am first implementing decision tree , because in decision tree they split the nodes recursively until only pure nodes are left i.e.
data belonging to the same groups are left on each side , data segregation is done here.The splitting of data is done based on the condition 
at the root node. This process keeps on happening until data is fully seperated according to their groups.Firstly, I am splitting the data with 20% 
using for testing and 80% for training purpose.Then I made a decision tree from scratch containing various functions.Next I implemented Naive Bayes 
Classifier with Gaussian modeling .It is a machine learning model with probablistic approach.I also implemented this from scratch. After defining both 
the classifiers , I evaluated both using stratified 10-fold cross-validation.Then I plotted comparison of both the classifiers.

In [9]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class Decision_Tree_Information_Gain:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_classes = len(np.unique(y))
        self.n_features = X.shape[1]
        self.tree = self.growth_of_tree(X, y)

    def growth_of_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        
        if (self.max_depth is not None and depth == self.max_depth) or n_labels == 1:
            return Node(value=self.Most_Common_Label(y))

        Best_Split = self.search_best_split(X, y)
        if Best_Split is None:
            return Node(value=self.Most_Common_Label(y))
        
        all_left_indices = np.where(X[:, Best_Split['feature']] <= Best_Split['threshold'])[0]
        all_right_indices = np.where(X[:, Best_Split['feature']] > Best_Split['threshold'])[0]

        Left_tree = self.growth_of_tree(X[all_left_indices], y[all_left_indices], depth + 1)
        Right_tree = self.growth_of_tree(X[all_right_indices], y[all_right_indices], depth + 1)

        return Node(feature=Best_Split['feature'], threshold=Best_Split['threshold'],
                    left=Left_tree, right=Right_tree)

    def search_best_split(self, X, y):
        Best_Split = None
        Best_Info_Gain = -1

        for feature in range(self.n_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                all_left_indices = np.where(X[:, feature] <= threshold)[0]
                all_right_indices = np.where(X[:, feature] > threshold)[0]

                if len(all_left_indices) > 0 and len(all_right_indices) > 0:
                    info_gain = self.Information_gained(y, y[all_left_indices], y[all_right_indices])
                    if info_gain > Best_Info_Gain:
                        Best_Split = {'feature': feature, 'threshold': threshold}
                        Best_Info_Gain = info_gain

        return Best_Split

    def Information_gained(self, parent, left_child, right_child):
        p = len(parent)
        pl = len(left_child)
        pr = len(right_child)
        
        return self.Entropy(parent) - (pl/p) * self.Entropy(left_child) - (pr/p) * self.Entropy(right_child)

    def Entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))

    def Most_Common_Label(self, y):
        labels, counts = np.unique(y, return_counts=True)
        return labels[np.argmax(counts)]

    def predict(self, X):
        return np.array([self.Prediction_Tree(x, self.tree) for x in X])

    def Prediction_Tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self.Prediction_Tree(x, node.left)
        else:
            return self.Prediction_Tree(x, node.right)

    def get_params(self, deep=True):
        return {'max_depth': self.max_depth}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

class Node:
    def __init__(self, feature=None, threshold=None, value=None, left=None, right=None):
        self.feature = feature
        self.threshold = threshold
        self.value = value
        self.left = left
        self.right = right

class Naive_Bayes_Classifier_Gaussian_Modeling:
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        self.means = np.zeros((len(self.classes), n_features))
        self.stds = np.zeros((len(self.classes), n_features))
        self.priors = np.zeros(len(self.classes))

        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.means[i] = X_c.mean(axis=0)
            self.stds[i] = X_c.std(axis=0)
            self.priors[i] = len(X_c) / n_samples

    def predict(self, X):
        probs = np.zeros((X.shape[0], len(self.classes)))
        for i, c in enumerate(self.classes):
            probs[:, i] = np.prod(self._gaussian_pdf(X, self.means[i], self.stds[i]), axis=1) * self.priors[i]
        return self.classes[np.argmax(probs, axis=1)]

    def _gaussian_pdf(self, X, mean, std):
        exponent = np.exp(-(X - mean) * 2 / (2 * std * 2))
        return (1 / (np.sqrt(2 * np.pi) * std)) * exponent

    def get_params(self, deep=True):
        return {}

def Evaluation_of_Classifier(clf, X, y):
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    f1_scores = cross_val_score(clf, X, y, cv=skf, scoring='f1_macro')
    return f1_scores

dt_classifier = Decision_Tree_Information_Gain()
nb_classifier = Naive_Bayes_Classifier_Gaussian_Modeling()

dt_f1_scores = Evaluation_of_Classifier(dt_classifier, X, y)
dt_mean_f1 = np.mean(dt_f1_scores)
dt_std_f1 = np.std(dt_f1_scores)

nb_f1_scores = Evaluation_of_Classifier(nb_classifier, X, y)
nb_mean_f1 = np.mean(nb_f1_scores)
nb_std_f1 = np.std(nb_f1_scores) 

classifiers = ['Decision Tree', 'Naive Bayes']
mean_f1_scores = [dt_mean_f1, nb_mean_f1]
std_f1_scores = [dt_std_f1, nb_std_f1]

plt.figure(figsize=(8, 6))
bars = plt.bar(classifiers, mean_f1_scores, yerr=std_f1_scores, capsize=5, color=['red', 'purple'])

for bar, mean_f1 in zip(bars, mean_f1_scores):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.02, f'{mean_f1:.2f}', ha='center', va='bottom')

plt.xlabel('Classifier')
plt.ylabel('F1 Score (Avg ± Std)')
plt.title('Performance Comparison of Classifiers')
plt.xticks(rotation=45)
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\EndUser\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\EndUser\AppData\Local\Temp\ipykernel_13728\2873932603.py", line 111, in fit
    X_c = X[y == c]
          ~^^^^^^^^
IndexError: boolean index did not match indexed array along dimension 1; dimension is 30 but corresponding boolean dimension is 1


In [ ]:
Question - 2  Dimensionality reduction with the Singular Value Decomposition
Here I am implementing dimensioanlity reduction using SVD to get the best axis to project on, meaning best means minimum sum of sqaures of 
projection error .For this implementation , I am using truncated SVD because after much research , I found out that Truncated SVD works very well 
with both sparse and dense data. Basically , here I did matrix multiplication X=UΣV(t). U is the matrix containing left singular vectors ,Σ is a matrix 
that contain top k values , through which we are eliminating the smallest singular value and V(t) is the tranpose of matrix V conitaning right singular
vectors.I have generated the Truncated SVD from scratch using different matrix related operations.We are eliminating left right most  column from U and 
bottom most row from V(t).

In [10]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

# Assuming you already have X and y loaded
# Example: X, y = make_classification(...)

# Splitting data into 80% train and 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Implementing truncated svd from scratch
def Truncated_Svd(X, k):
    X_centered = X - np.mean(X, axis=0)
    Covariance_Matrix = np.dot(X_centered.T, X_centered)
    eigenvalues, eigenvectors = np.linalg.eigh(Covariance_Matrix)
    Top_k_Indices = np.argsort(eigenvalues)[::-1][:k]
    U = eigenvectors[:, Top_k_Indices]
    S = np.sqrt(np.abs(eigenvalues[Top_k_Indices]))
    Vt = np.dot(U.T, X_centered.T) / S[:, np.newaxis]
    return U, S, Vt

# Applying SVD for dimensionality reduction
def Applying_SVD(X_train, X_test, rank):
    U_train, _, Vt_train = Truncated_Svd(X_train, rank)
    X_train_reduced = np.dot(X_train, U_train)
    X_test_reduced = np.dot(X_test, U_train)
    return X_train_reduced, X_test_reduced

class Decision_Tree_Information_Gain:
    def __init__(self, max_depth=None): 
        self.max_depth = max_depth
        
    def fit(self, X, y):
        self.n_classes = len(np.unique(y))
        self.n_features = X.shape[1]
        self.tree = self.growth_of_tree(X, y)
        
    def growth_of_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        if (self.max_depth is not None and depth == self.max_depth) or n_labels == 1:
            return Node(value=self.Most_Common_Label(y))
        Best_Split = self.search_best_split(X, y)
        if Best_Split is None:
            return Node(value=self.Most_Common_Label(y))
        all_left_indices = np.where(X[:, Best_Split['feature']] <= Best_Split['threshold'])[0]
        all_right_indices = np.where(X[:, Best_Split['feature']] > Best_Split['threshold'])[0]
        Left_tree = self.growth_of_tree(X[all_left_indices], y[all_left_indices], depth + 1)
        Right_tree = self.growth_of_tree(X[all_right_indices], y[all_right_indices], depth + 1)
        return Node(feature=Best_Split['feature'], threshold=Best_Split['threshold'], left=Left_tree, right=Right_tree)
        
    def search_best_split(self, X, y):
        Best_Split = None
        Best_Info_Gain = -1
        for feature in range(self.n_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                all_left_indices = np.where(X[:, feature] <= threshold)[0]
                all_right_indices = np.where(X[:, feature] > threshold)[0]
                if len(all_left_indices) > 0 and len(all_right_indices) > 0:
                    info_gain = self.Information_gained(y, y[all_left_indices], y[all_right_indices])
                    if info_gain > Best_Info_Gain:
                        Best_Split = {'feature': feature, 'threshold': threshold}
                        Best_Info_Gain = info_gain
        return Best_Split

    def Information_gained(self, parent, left_child, right_child):
        p = len(parent)
        pl = len(left_child)
        pr = len(right_child)
        return self.Entropy(parent) - (pl/p) * self.Entropy(left_child) - (pr/p) * self.Entropy(right_child)

    def Entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))

    def Most_Common_Label(self, y):
        labels, counts = np.unique(y, return_counts=True)
        return labels[np.argmax(counts)]
    
    def predict(self, X):
        return np.array([self.Prediction_Tree(x, self.tree) for x in X])

    def Prediction_Tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self.Prediction_Tree(x, node.left)
        else:
            return self.Prediction_Tree(x, node.right)
            
    def get_params(self, deep=True):
        return {'max_depth': self.max_depth}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

class Node:
    def __init__(self, feature=None, threshold=None, value=None, left=None, right=None):
        self.feature = feature
        self.threshold = threshold
        self.value = value
        self.left = left
        self.right = right

class Naive_Bayes_Classifier_Gaussian_Modeling:

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_indices = {cls: idx for idx, cls in enumerate(self.classes)}
        self.class_priors = np.array([np.mean(y == c) for c in self.classes])
        self.class_means = [np.mean(X[y == c], axis=0) for c in self.classes]
        self.class_vars = [np.var(X[y == c], axis=0) for c in self.classes]

    def Gaussian_Density(self, x, mean, var):
        return (1 / np.sqrt(2 * np.pi * var)) * np.exp(-(x - mean)**2 / (2 * var))

    def Predicting_Class_Probas(self, x):
        class_probs = []
        for cls in self.classes:
            idx = self.class_indices[cls]
            prior = self.class_priors[idx]
            likelihood = np.prod(self.Gaussian_Density(x, self.class_means[idx], self.class_vars[idx]))
            class_probs.append(prior * likelihood)
        return class_probs / np.sum(class_probs)

    def predict(self, X):
        return np.array([self.classes[np.argmax(self.Predicting_Class_Probas(x))] for x in X])

    def get_params(self, deep=True):
        return {}

    def set_params(self, **parameters):
        return self

# Listing of SVD ranks for evaluation in the line graph.
svd_ranks = [2, 4, 6, 8, 10]

# Dictionary to store the performance of both the classifiers.
classifier_results = {'Decision Tree': [], 'Naive Bayes': []}

# Evaluation of both the classifiers for each SVD rank.
for rank in svd_ranks:
    X_train_reduced, X_test_reduced = Applying_SVD(X_train, X_test, rank)
    
    dt_classifier = Decision_Tree_Information_Gain()
    nb_classifier = Naive_Bayes_Classifier_Gaussian_Modeling()
    
    # Fitting of decision Tree Classifier
    dt_classifier.fit(X_train_reduced, y_train)
    dt_predictions = dt_classifier.predict(X_test_reduced)
    dt_mean_f1 = f1_score(y_test, dt_predictions, average='weighted')
    
    # Fitting of Naive Bayes Classifier
    nb_classifier.fit(X_train_reduced, y_train)
    nb_predictions = nb_classifier.predict(X_test_reduced)
    nb_mean_f1 = f1_score(y_test, nb_predictions, average='weighted')
    
    # Storing of results
    classifier_results['Decision Tree'].append(dt_mean_f1)
    classifier_results['Naive Bayes'].append(nb_mean_f1)

# Plotting the comparison of performance of both classifiers in the line graph.
plt.figure(figsize=(8, 4))
for classifier, results in classifier_results.items():
    plt.plot(svd_ranks, results, label=classifier, marker='o')

plt.xlabel('SVD Rank')
plt.ylabel('F1 Score')
plt.title('Performance Comparison of Classifiers with SVD')
plt.xticks(svd_ranks)
plt.legend()
plt.grid(True)
plt.show()

IndexError: boolean index did not match indexed array along dimension 1; dimension is 2 but corresponding boolean dimension is 1

In [ ]:
Question - 3  Feature selection with randomization : 
In this question , I implemented feature selection technique with randomization to determine which features are of more important by measuring the 
model performance by k-fold cross validaition.The features are selected are random here and the values are permuted here without making any changes in
other features.In this way , an importance of feature can be determines by comparions of model with or without that selected feature.Also the importance 
of any specific feature is found out by fidning degree of performance degradation cause by the randomization of selected feature.

In [11]:
classifiers = {
    'Decision Tree': Decision_Tree_Information_Gain(),
    'Naive Bayes': Naive_Bayes_Classifier_Gaussian_Modeling()
}

# Training and evaluation function
def Training_AND_Evaluatation_Model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred, pos_label='M')  # Specify pos_label

# Create a sample split of the data
X_fs_sample, X_holdout, y_fs_sample, y_holdout = train_test_split(X, y, test_size=0.8, stratify=y, random_state=42)

# Dictionary for storing performance and feature importance
Importance_Feature_Dictionary = {}
top_feature_performance_dict = {}

# Calculate classifier performance and feature importance
for model_name, model in classifiers.items():
    feature_importance = []

    # Iterate through each feature
    for feature_idx in range(X_fs_sample.shape[1]):
        X_fs_randomized = X_fs_sample.copy()
        np.random.shuffle(X_fs_randomized[:, feature_idx])

        # Cross-validation setup
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        f1_actual_scores = []
        f1_randomized_scores = []

        # Perform cross-validation
        for train_idx, test_idx in cv.split(X_fs_sample, y_fs_sample):
            X_train_fold, X_test_fold = X_fs_sample[train_idx], X_fs_sample[test_idx]
            y_train_fold, y_test_fold = y_fs_sample[train_idx], y_fs_sample[test_idx]

            # Train and evaluate with actual data
            f1_actual = Training_AND_Evaluatation_Model(model, X_train_fold, y_train_fold, X_test_fold, y_test_fold)
            f1_actual_scores.append(f1_actual)

            # Train and evaluate with randomized feature data
            X_train_fold_randomized = X_train_fold.copy()
            X_train_fold_randomized[:, feature_idx] = np.random.permutation(X_train_fold[:, feature_idx])
            f1_randomized = Training_AND_Evaluatation_Model(model, X_train_fold_randomized, y_train_fold, X_test_fold, y_test_fold)
            f1_randomized_scores.append(f1_randomized)

        # Calculate average F1 scores
        avg_f1_actual = np.mean(f1_actual_scores)
        avg_f1_randomized = np.mean(f1_randomized_scores)

        # Calculate performance drop percentage
        percentage_drop = ((avg_f1_actual - avg_f1_randomized) / avg_f1_actual) * 100

        # Store performance drop percentage
        feature_importance.append((feature_idx, percentage_drop))

    # Sort features by importance
    feature_importance.sort(key=lambda x: x[1], reverse=True)
    Importance_Feature_Dictionary[model_name] = feature_importance

    # Extract feature names and importance values for plotting
    feature_names = [f'feature_{idx}' for idx, _ in feature_importance]
    importance = [importance for _, importance in feature_importance]

    # Store feature names and importance values
    Importance_Feature_Dictionary[model_name] = (feature_names, importance)

    # Evaluate classifier performance using top 1 to top 10 features
    top_feature_performance = []
    
    for num_features in range(1, 11):
        top_features = [feature[0] for feature in feature_importance[:num_features]]
        X_holdout_selected = X_holdout[:, top_features]

        # Train and evaluate model with top selected features
        f1_holdout = Training_AND_Evaluatation_Model(model, X_fs_sample[:, top_features], y_fs_sample, X_holdout_selected, y_holdout)
        top_feature_performance.append(f1_holdout)

    top_feature_performance_dict[model_name] = top_feature_performance

# Plot combined feature importance bar chart for both classifiers
plt.figure(figsize=(8, 4))
dt_feature_names = Importance_Feature_Dictionary['Decision Tree'][0][:10]
dt_importance = Importance_Feature_Dictionary['Decision Tree'][1][:10]

# Get the top 10 features and their importance for Naive Bayes
nb_feature_names = Importance_Feature_Dictionary['Naive Bayes'][0][:10]
nb_importance = Importance_Feature_Dictionary['Naive Bayes'][1][:10]

bar_width = 0.35
index = np.arange(10)

# Bar chart for Decision Tree classifier
plt.bar(index, dt_importance, bar_width, label='Decision Tree')

# Bar chart for Naive Bayes classifier
plt.bar(index + bar_width, nb_importance, bar_width, label='Naive Bayes')

plt.xlabel('Top 10 Feature Names')
plt.ylabel('Importance (Percentage Drop in F1 Score)')
plt.title('Top 10 Feature Importance')
plt.xticks(index + bar_width / 2, dt_feature_names, rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.show()

# Plot combined classifier performance with top features line graph
plt.figure(figsize=(8, 4))

# Line graph for Decision Tree
plt.plot(range(1, 11), top_feature_performance_dict['Decision Tree'], marker='o', label='Decision Tree')

# Line graph for Naive Bayes
plt.plot(range(1, 11), top_feature_performance_dict['Naive Bayes'], marker='o', label='Naive Bayes')

plt.xlabel('Number of Top Features')
plt.ylabel('F1 Score')
plt.title('Classifier Performance with Top Features')
plt.xticks(range(1, 11))
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

KeyboardInterrupt: 